In [1]:
from py2neo import Graph
import numpy as np 
from pandas import DataFrame
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
import pandas as pd
import plotly 
import plotly.graph_objs as go
import qgrid
from scipy import stats
from sklearn.cluster.bicluster import SpectralBiclustering

local_connection_url = "http://localhost:7474/db/data"
connection_to_graph = Graph(local_connection_url)

# please add your plotly api credentials to plotly_config in your own machine. Visit https://plot.ly/python/getting-started/
plotly_config = json.load(open('plotly_config.json'))
plotly.tools.set_credentials_file(username=plotly_config['username'], api_key=plotly_config['key'])


org_query = """ MATCH (n:Owner) 
                    WITH n.name AS Organization 
                    RETURN Organization;
                    """

org_names = list(set(DataFrame(connection_to_graph.data(org_query)).as_matrix()[:, 0]))
org_names.sort()

print 'The org list has {} organizations.'.format(len(org_names))

The org list has 14806 organizations.


In [8]:
good_ones = []
count = 0
record_threshold = 20

for organization in org_names:
    
    print 'Processing org {} / {}. And we have {} good. \r'.format(count + 1, len(org_names), len(good_ones)),
    
    # get data
    country_no_interestions = """   MATCH (a:Asset)-[:CONTAINS]->(fs:Feedstock)
                                    MATCH (a:Asset)-[:CONTAINS]->(out:Output)
                                    MATCH (a:Asset)-[:CONTAINS]->(pt:ProcessingTech)
                                    WHERE a.owner = "{}"
                                    RETURN fs.term, pt.term, out.term, count(a)
                                    """.format(organization.encode('utf-8'))
    data = DataFrame(connection_to_graph.data(country_no_interestions)).as_matrix()
    
    
    if data.size != 0:
        number_of_records = np.sum(data[:, 0])
        
        if number_of_records >= record_threshold:
            good_ones.append(organization)

    count += 1
    
np.save(file='Data/Good_organizations.npy', arr=np.asarray(good_ones))    

In [ ]:
process_variables = ['Feedstock', 'Output', 'ProcessingTech']

country_intersections = """     MATCH (a:Asset)-[:CONTAINS]->(fs:{})
                                MATCH (a:Asset)-[:CONTAINS]->(t:{})
                                WHERE fs<>t AND a.country = "{}"
                                RETURN fs.term, t.term, count(a)
                                """

In [9]:
a = np.load(file='Data/Good_organizations.npy')
print a

[u'AALBORG UNIV' u'ABELLON CLEANENERGY LTD'
 u'ABENGOA BIOENERGIA NUEVAS TECNOLOGIAS SA' u'ABENGOA BIOENERGY'
 u'ABENGOA BIOENERGY BIOMASS OF KANSAS LLC'
 u'ABENGOA BIOENERGY NEW TECHNOLOGIES INC' u'AGR & AGRI FOOD CANADA'
 u'AGRIVIDA INC'
 u'AMITY INSTITUTE OF BIOTECHNOLOGY, AMITY UNIVERSITY UTTAR PRADESH, SECTOR 125'
 u'ANNA UNIV' u'ARCHER-DANIELS MIDLAND CO' u'ARGONNE NATL LAB'
 u'ARISTOTLE UNIV THESSALONIKI' u'ARS' u'ASTON UNIV'
 u'BEIJING FORESTRY UNIV' u'BEIJING UNIV CHEM TECHNOL'
 u'BRIGHAM YOUNG UNIV' u'CAPITAL NORMAL UNIV' u'CATCHLIGHT ENERGY LLC'
 u'CHINA AGR UNIV' u'CHINA UNIV PETR' u'CHINESE ACAD SCI'
 u'CHONBUK NATL UNIV' u'CHONNAM NATL UNIV' u'CHUNGBUK NATL UNIV'
 u'CITY UNIV HONG KONG' u'CONSEJO NACL INVEST CIENT & TECN' u'COSKATA'
 u'CSIC' u'CSIR' u'CTR BIOTECHNOL SFAX' u'DCA CONSULTANTS LTD'
 u'DEPARTMENT OF BIOPROCESS TECHNOLOGY, FACULTY OF BIOTECHNOLOGY AND BIOMOLECULAR SCIENCES, UNIVERSITI PUTRA MALAYSIA (UPM)'
 u'DEUTSCH BIOMASSEFORSCHUNGSZENTRUM GEMEINNUTZIGE G' u